In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

df_raw = pd.read_csv('/content/drive/MyDrive/Brazilian_Customer.csv')
#df_raw = pd.read_csv('/content/drive/MyDrive/분석미니플젝6조/Brazilian_Customer.csv')
print("결측치 확인")
print(df_raw.isnull().sum())

결측치 확인
order_id                              0
customer_id                           0
order_status                          0
order_purchase_timestamp              0
order_approved_at                    15
order_delivered_carrier_date       1235
order_delivered_customer_date      2471
order_estimated_delivery_date         0
review_id                             0
review_score                          0
review_comment_title             103437
review_comment_message            67650
review_creation_date                  0
review_answer_timestamp               0
payment_sequential                    0
payment_type                          0
payment_installments                  0
payment_value                         0
customer_unique_id                    0
customer_zip_code_prefix              0
customer_city                         0
customer_state                        0
order_item_id                         0
product_id                            0
seller_id                        

In [ ]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117329 entries, 0 to 117328
Data columns (total 39 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       117329 non-null  object 
 1   customer_id                    117329 non-null  object 
 2   order_status                   117329 non-null  object 
 3   order_purchase_timestamp       117329 non-null  object 
 4   order_approved_at              117314 non-null  object 
 5   order_delivered_carrier_date   116094 non-null  object 
 6   order_delivered_customer_date  114858 non-null  object 
 7   order_estimated_delivery_date  117329 non-null  object 
 8   review_id                      117329 non-null  object 
 9   review_score                   117329 non-null  int64  
 10  review_comment_title           13892 non-null   object 
 11  review_comment_message         49679 non-null   object 
 12  review_creation_date          

In [ ]:
# # 결측치 제거
# df_raw = df_raw.drop(['review_comment_title', 'review_comment_message'], axis = 1)
# df_raw = df_raw.dropna(axis = 0)

In [ ]:
df = df_raw.copy()

#### 리뷰 관련 파생변수

In [ ]:
# 리뷰 메세지 길이 변수(포르투갈ver) : review_comment_message_len
review_comment_message_len = []

for m in df['review_comment_message']:
  if (type(m) is float):
    review_comment_message_len.append(0)
  else:
    word = list(m.split())
    review_comment_message_len.append(len(word))

df['review_comment_message_len'] = review_comment_message_len

In [ ]:
#!pip install googletrans==4.0.0-rc1

포르투갈어 -> 영어 번역 후 진행

In [ ]:
# #포르투갈어 -> 영어 번역
# import googletrans
# import time
# import random

# translator = googletrans.Translator()
# trans = []

# for m in df['review_comment_message']:
#   try:
#     trans.append(translator.translate(m, src='pt', dest='en').text)
#     time.sleep(1)
#   except IndexError:
#     trans.append('')

In [ ]:
# len(trans)

너무 오래걸려서 ,,, 약 5천개까지만 번역

In [ ]:
# 결측치 제거
df = df.drop(['review_comment_title', 'review_comment_message'], axis = 1)
df = df.dropna(axis = 0)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113216 entries, 0 to 117328
Data columns (total 38 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       113216 non-null  object 
 1   customer_id                    113216 non-null  object 
 2   order_status                   113216 non-null  object 
 3   order_purchase_timestamp       113216 non-null  object 
 4   order_approved_at              113216 non-null  object 
 5   order_delivered_carrier_date   113216 non-null  object 
 6   order_delivered_customer_date  113216 non-null  object 
 7   order_estimated_delivery_date  113216 non-null  object 
 8   review_id                      113216 non-null  object 
 9   review_score                   113216 non-null  int64  
 10  review_creation_date           113216 non-null  object 
 11  review_answer_timestamp        113216 non-null  object 
 12  payment_sequential            

#### 배송 관련 파생변수

In [ ]:
df["order_purchase_timestamp"] = pd.to_datetime(df_raw["order_purchase_timestamp"], format='%Y-%m-%d %H:%M:%S')
df["order_delivered_customer_date"] = pd.to_datetime(df_raw["order_delivered_customer_date"], format='%Y-%m-%d %H:%M:%S')
df["order_estimated_delivery_date"] = pd.to_datetime(df_raw["order_estimated_delivery_date"], format='%Y-%m-%d %H:%M:%S')

In [ ]:
df['review_answer_timestamp'] = pd.to_datetime(df_raw['review_answer_timestamp'], format='%Y-%m-%d %H:%M:%S')
df['review_creation_date'] = pd.to_datetime(df_raw['review_creation_date'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
# 배송 기간 변수 : delivery_period

df['delivery_period'] = df['order_purchase_timestamp'] - df['order_delivered_customer_date']

In [ ]:
# 조기 배송 변수 : early_delivery

df['early_delivery'] = df['order_estimated_delivery_date'] - df['order_delivered_customer_date']

In [ ]:
import numpy as np

early_delivery_div = []

for d in df['early_delivery']:
  if(d.days < 0) :
    early_delivery_div.append(-1)
  elif (d.days == 0) :
    early_delivery_div.append(0)
  else :
    early_delivery_div.append(1)

df['early_delivery_div'] = early_delivery_div

In [ ]:
# 조기 배송 여부 변수 : early_delivery_div

df['early_delivery_div'].value_counts()

 1    102870
-1      8715
 0      1631
Name: early_delivery_div, dtype: int64

In [ ]:
# 리뷰 작성 시간 변수 : review_time

df['review_time'] = df['review_answer_timestamp'] - df['review_creation_date']
df.review_time.value_counts()

2 days 22:17:55    63
0 days 21:08:44    38
5 days 18:49:50    26
4 days 13:20:31    25
1 days 08:34:13    24
                   ..
0 days 16:04:01     1
1 days 06:21:25     1
0 days 14:15:15     1
1 days 03:39:59     1
0 days 15:14:23     1
Name: review_time, Length: 78941, dtype: int64

#### 가격 관련 파생 변수

#####소득 수준 변수

In [ ]:
income_level = []

for p in df['payment_installments']:
  if(p >= 0 and p < 8):
    income_level.append(2)
  elif(p >= 8 and p <16):
    income_level.append(1)
  else:
    income_level.append(0)
  
df['income_level'] = income_level

In [ ]:
df['income_level'].value_counts(0)

2    100591
1     12512
0       113
Name: income_level, dtype: int64

##### 카테고리별 가격 변수 : cat_cheap

In [ ]:
cat = df.groupby(['product_category_name'])

In [ ]:
cat = pd.DataFrame(cat['price'].quantile(0.25))

In [ ]:
cat = cat.rename(columns = {'price':'price_0.25'})

In [ ]:
df = pd.merge(df,cat, how='left',on='product_category_name')

In [ ]:
df['cat_cheap'] = 0
df.loc[df['price'] < df['price_0.25'], 'cat_cheap'] = 1

In [ ]:
df.cat_cheap.value_counts()

0    85888
1    27328
Name: cat_cheap, dtype: int64

In [ ]:
df = df.drop(['price_0.25'], axis = 1)

#### 지역 관련 파생변수

##### 지역 구분 : cu_region

In [ ]:
df.loc[df[df['customer_state'].isin(['RR','PA','AP','AM','TO','AC','RO'])].index, 'cu_region'] = 'north'
df.loc[df[df['customer_state'].isin(['MA','PE','PI','CE','RN','PB','AL','SE','BA'])].index, 'cu_region'] = 'north east'
df.loc[df[df['customer_state'].isin(['MT','GO','DF','MS'])].index, 'cu_region'] = 'mid west'
df.loc[df[df['customer_state'].isin(['MG','ES','RJ','SP'])].index, 'cu_region'] = 'south east'
df.loc[df[df['customer_state'].isin(['PR','SC','RS'])].index, 'cu_region'] = 'south'

In [ ]:
df.cu_region.value_counts()

south east    77880
south         16245
north east    10447
mid west       6599
north          2045
Name: cu_region, dtype: int64

##### 발전도 낮은 지역 파생변수 : low_develop

In [ ]:
df['low_develop'] = 0
df.loc[df[df.cu_region.isin(['north','north east'])].index, 'low_develop'] = 1

In [ ]:
df.low_develop.value_counts()

0    100724
1     12492
Name: low_develop, dtype: int64

#### 카테고리 파생변수

In [ ]:
# 카테고리 대분류 변수 : 분류 기준은 노션 > 파생변수 생성 > 카테고리 분류 참고!

def switch(x):
  return {
      'cama_mesa_banho' : 0,
      'beleza_saude' : 8,
      'esporte_lazer' : 8,
      'moveis_decoracao' : 3,
      'informatica_acessorios' : 4,
      'utilidades_domesticas' : 2,
      'relogios_presentes' : 6,
      'telefonia' : 4,
      'ferramentas_jardim' : 2,
      'automotivo' : 5,
      'brinquedos' : 6,
      'cool_stuff' : 0,
      'perfumaria' : 0,
      'bebes' : 1,
      'eletronicos' : 4,
      'papelaria' : 6,
      'fashion_bolsas_e_acessorios' : 0,
      'pet_shop' : 7,
      'moveis_escritorio' : 3,
      'consoles_games' : 4,
      'malas_acessorios' : 0,
      'construcao_ferramentas_construcao' : 3,
      'eletrodomesticos' : 4,
      'instrumentos_musicais' : 6,
      'eletroportateis' : 4,
      'casa_construcao' : 3,
      'livros_interesse_geral' : 6,
      'moveis_sala' : 3,
      'alimentos' : 2,
      'casa_conforto' : 2,
      'bebidas' : 2,
      'audio' : 4,
      'market_place' : 9,
      'construcao_ferramentas_iluminacao' : 3,
      'moveis_cozinha_area_de_servico_jantar_e_jardim' : 3,
      'climatizacao' : 4,
      'alimentos_bebidas' : 2,
      'fashion_calcados' : 0,
      'telefonia_fixa' : 4,
      'industria_comercio_e_negocios' : 9,
      'livros_tecnicos' : 6,
      'eletrodomesticos_2' : 4,
      'agro_industria_e_comercio' : 9,
      'construcao_ferramentas_jardim' : 9,
      'artes' : 0,
      'pcs' : 9,
      'sinalizacao_e_seguranca' : 9,
      'construcao_ferramentas_seguranca' : 3,
      'artigos_de_natal' : 0,
      'fashion_roupa_masculina' : 0,
      'fashion_underwear_e_moda_praia' : 0,
      'moveis_quarto' : 3,
      'construcao_ferramentas_ferramentas' : 3,
      'tablets_impressao_imagem' : 4,
      'portateis_casa_forno_e_cafe' : 2,
      'cine_foto' : 6,
      'dvds_blu_ray' : 6,
      'livros_importados' : 6,
      'fashion_roupa_feminina' : 0,
      'artigos_de_festas' : 2,
      'moveis_colchao_e_estofado' : 2,
      'musica' : 9,
      'fraldas_higiene' : 2,
      'fashion_esporte' : 0,
      'flores' : 2,
      'casa_conforto_2' : 2,
      'artes_e_artesanato' : 0,
      'portateis_cozinha_e_preparadores_de_alimentos' : 2,
      'la_cuisine' : 2,
      'cds_dvds_musicais' : 6,
      'pc_gamer' : 4,
      'fashion_roupa_infanto_juvenil' : 0,
      'seguros_e_servicos' : 9
  }.get(x, 9)

In [ ]:
product_main_category = []

for c in df['product_category_name']:
  product_main_category.append(switch(c))

df['product_main_category'] = product_main_category

#### 재구매 관련 파생변수

##### 재구매율 파생변수 : re_percent

In [ ]:
group = df.groupby(['seller_id'])

In [ ]:
d = group.customer_unique_id.nunique() # 스토어별 구매고객

In [ ]:
group2 = df.groupby(['seller_id','customer_unique_id'])

In [ ]:
a = (group2.nunique().order_id > 2).sum(level = 0) # 각 스토어별 재구매한 고객의 수

<ipython-input-103-5865e056b5b8>:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  a = (group2.nunique().order_id > 2).sum(level = 0) # 각 스토어별 재구매한 고객의 수


In [ ]:
re = pd.DataFrame(a / d)

In [ ]:
re.rename(columns = {0:'re_percent'}, inplace = True)

In [ ]:
df = pd.merge(df,re, how='left',on='seller_id')

##### 재 구매 여부 파생변수 : re_ox

In [ ]:
df['re_ox'] = 0
df.loc[df.re_percent != 0, 're_ox'] = 1 

In [ ]:
df.re_ox.value_counts()

0    101827
1     11389
Name: re_ox, dtype: int64

#### 상품 설명 관련 파생변수

In [ ]:
# 상품 정보 전달 정도를 파악할 수 있는 변수 : product_info

df['product_info'] = (df['product_photos_qty'] + df['product_description_lenght']) /2

#### 상품 크기 관련 파생변수

In [ ]:
# 부피 변수 (사각형 부피 공식 사용-박스) : product_size

df['product_size'] = df['product_length_cm'] * df['product_width_cm'] * df['product_height_cm']

#### 판매자 관련 파생변수

참고 : https://www.kaggle.com/code/khakim17/custumer-satisfaction-prediction-and-analysist#CREATING-ANOTHER-COLUMNS-THAT-NEEDED

In [ ]:
# 판매자 등급, 상품 등급, 특정 판매자가 판 상품 개수 변수 (SELLER RATING, PRODUCT RATING AND NUMBER OF PRODUCT SOLD BY SELLER)

In [ ]:
# product_soldx = {}

# for item in df['seller_id'].unique():
#   product_soldx[item] = int(df[df['seller_id'] == item].describe().iloc[0][0])

# product_sold = []
# for item in df['seller_id']:
#   product_sold.append(product_soldx[item])

# df['product_sold'] = product_sold

In [ ]:
# df['seller_rating']

역시 시간이 오래 걸려서 seller_rating이랑 product_solidx 변수까지만 생성하는 걸로..!

#### 주문건수 관련 변수

##### 스토어 주문건수 변수 : seller_popularity

In [ ]:
group = df.groupby(['seller_id'])

In [ ]:
store_popularity = group.order_id.nunique()

In [ ]:
store_popularity

seller_id
0015a82c2db000af6aaaf3ae2ecb0532      3
001cca7ae9ae17fb1caed9dfb1094831    192
002100f778ceb8431b7a1020ff7ab48f     50
004c9cd9d87a3c30c522c48c4fc07416    142
00720abe85ba0859807595bbf045a33b     13
                                   ... 
ffc470761de7d0232558ba5e786e57b7     27
ffdd9f82b9a447f6f8d4b91554cc7dd3     18
ffeee66ac5d5a62fe688b9d26f83f534     14
fffd5413c0700ac820c7069d66d98c89     56
ffff564a4f9085cd26170f4732393726      8
Name: order_id, Length: 2909, dtype: int64

In [ ]:
store_popularity = pd.DataFrame(store_popularity)
store_popularity.rename(columns = {'order_id':'seller_popularity'}, inplace = True)

In [ ]:
store_popularity

,seller_popularity
seller_id,
0015a82c2db000af6aaaf3ae2ecb0532,3
001cca7ae9ae17fb1caed9dfb1094831,192
002100f778ceb8431b7a1020ff7ab48f,50
004c9cd9d87a3c30c522c48c4fc07416,142
00720abe85ba0859807595bbf045a33b,13
...,...
ffc470761de7d0232558ba5e786e57b7,27
ffdd9f82b9a447f6f8d4b91554cc7dd3,18
ffeee66ac5d5a62fe688b9d26f83f534,14


In [ ]:
df = pd.merge(df,store_popularity, how='left',on='seller_id')

##### 고객 주문건수 변수 : 'cu_buy'

In [ ]:
cu_group = df.groupby(['customer_unique_id'])

In [ ]:
cu_buy = cu_group.order_id.nunique()

In [ ]:
cu_buy = pd.DataFrame(cu_buy)
cu_buy.rename(columns = {'order_id':'cu_buy'}, inplace = True)

In [ ]:
df = pd.merge(df,cu_buy, how='left',on='customer_unique_id')

#### 확인해보기

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113216 entries, 0 to 113215
Data columns (total 53 columns):
 #   Column                         Non-Null Count   Dtype          
---  ------                         --------------   -----          
 0   order_id                       113216 non-null  object         
 1   customer_id                    113216 non-null  object         
 2   order_status                   113216 non-null  object         
 3   order_purchase_timestamp       113216 non-null  datetime64[ns] 
 4   order_approved_at              113216 non-null  object         
 5   order_delivered_carrier_date   113216 non-null  object         
 6   order_delivered_customer_date  113216 non-null  datetime64[ns] 
 7   order_estimated_delivery_date  113216 non-null  datetime64[ns] 
 8   review_id                      113216 non-null  object         
 9   review_score                   113216 non-null  int64          
 10  review_creation_date           113216 non-null  datetime

In [ ]:
# df = df.drop(['product_sold', 'seller_rating'], axis = 1)

In [ ]:
# df.loc[:,['seller_popularity','product_sold']]  # product_sold랑 seller_popularity랑 의도한 건 같은 것 같은데 값이 다르게 나온다...

In [ ]:
# df['payment_type'].value_counts()

credit_card    83545
boleto         22022
voucher         6023
debit_card      1626
Name: payment_type, dtype: int64

#### 파생변수 생성한 열 제거

In [ ]:
#df = df.drop(['order_purchase_timestamp',
#              'order_delivered_customer_date',
#              'order_estimated_delivery_date',
#              'payment_installments',
#              'customer_state',
#              'product_category_name',
#              'product_description_lenght',
#              'product_photos_qty',
#              'product_length_cm','product_width_cm','product_height_cm','review_creation_date','review_answer_timestamp' ], axis = 1)

In [ ]:
# 중복되는 변수 제거
#df = df.drop(['customer_zip_code_prefix','seller_zip_code_prefix'], axis = 1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113216 entries, 0 to 113215
Data columns (total 53 columns):
 #   Column                         Non-Null Count   Dtype          
---  ------                         --------------   -----          
 0   order_id                       113216 non-null  object         
 1   customer_id                    113216 non-null  object         
 2   order_status                   113216 non-null  object         
 3   order_purchase_timestamp       113216 non-null  datetime64[ns] 
 4   order_approved_at              113216 non-null  object         
 5   order_delivered_carrier_date   113216 non-null  object         
 6   order_delivered_customer_date  113216 non-null  datetime64[ns] 
 7   order_estimated_delivery_date  113216 non-null  datetime64[ns] 
 8   review_id                      113216 non-null  object         
 9   review_score                   113216 non-null  int64          
 10  review_creation_date           113216 non-null  datetime

In [ ]:
# datetime으로 바꾸기- order_approved_at ,order_delivered_carrier_date,,shipping_limit_date
#dt_lst = ['order_approved_at','order_delivered_carrier_date','shipping_limit_date']

#for dt in dt_lst:
#    df[dt] = pd.to_datetime(df[dt], format='%Y-%m-%d %H:%M:%S')

In [ ]:
dt_lst = ['order_purchase_timestamp','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date',
         'order_estimated_delivery_date','review_creation_date','review_answer_timestamp','shipping_limit_date']

for dt in dt_lst:
    df[dt] = pd.to_datetime(df[dt], format='%Y-%m-%d %H:%M:%S')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113216 entries, 0 to 113215
Data columns (total 53 columns):
 #   Column                         Non-Null Count   Dtype          
---  ------                         --------------   -----          
 0   order_id                       113216 non-null  object         
 1   customer_id                    113216 non-null  object         
 2   order_status                   113216 non-null  object         
 3   order_purchase_timestamp       113216 non-null  datetime64[ns] 
 4   order_approved_at              113216 non-null  datetime64[ns] 
 5   order_delivered_carrier_date   113216 non-null  datetime64[ns] 
 6   order_delivered_customer_date  113216 non-null  datetime64[ns] 
 7   order_estimated_delivery_date  113216 non-null  datetime64[ns] 
 8   review_id                      113216 non-null  object         
 9   review_score                   113216 non-null  int64          
 10  review_creation_date           113216 non-null  datetime

In [ ]:
# 데이터 저장

df.to_csv('/content/drive/MyDrive/derived_variable.csv', index = False)
#df.to_csv('/content/drive/MyDrive/BOAZ-Analysis/미니 프로젝트1/derived_variable.csv', index = False)

#### NLP

In [ ]:
df_nlp = df.copy()

In [ ]:
df_nlp = df_nlp.iloc[:5300, :]

In [ ]:
df_nlp['review_eng'] = trans

df_nlp.head(5)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_id,review_score,...,seller_state,delivery_period,early_delivery,early_delivery_div,income_level,product_size,product_info,review_comment_message_len,product_main_category,review_eng
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,a54f0611adc9ed256b57ede6b6eb5114,4,...,SP,-9 days +13:31:20,7 days 02:34:47,1,2,1976.0,136.0,32,2,"I haven't tested the product yet, but it came ..."
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,a54f0611adc9ed256b57ede6b6eb5114,4,...,SP,-9 days +13:31:20,7 days 02:34:47,1,2,1976.0,136.0,32,2,"I haven't tested the product yet, but it came ..."
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,a54f0611adc9ed256b57ede6b6eb5114,4,...,SP,-9 days +13:31:20,7 days 02:34:47,1,2,1976.0,136.0,32,2,"I haven't tested the product yet, but it came ..."
3,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,delivered,2017-08-15 18:29:31,2017-08-15 20:05:16,2017-08-17 15:28:33,2017-08-18 14:44:43,2017-08-28,b46f1e34512b0f4c74a72398b03ca788,4,...,SP,-3 days +03:44:48,9 days 09:15:17,1,2,1976.0,136.0,15,2,They should pack the product better.The box ca...
4,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,delivered,2017-08-02 18:24:47,2017-08-02 18:43:15,2017-08-04 17:35:43,2017-08-07 18:30:01,2017-08-15,dc90f19c2806f1abba9e72ad3c350073,5,...,SP,-6 days +23:54:46,7 days 05:29:59,1,2,1976.0,136.0,12,2,"I only found her small for six cups, but it's ..."


In [ ]:
# 영문자 외 공백으로 변환
import re

for r in df_nlp['review_eng']:
  r = re.sub('[^a-zA-Z]', ' ', r)

In [ ]:
# 대문자 => 소문자 변환 + split

low = []

for r in df_nlp['review_eng']:
  low.append(r.lower().split())

In [ ]:
df_nlp['review_split'] = low

In [ ]:
# 불용어 제거
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


stops = set(stopwords.words('english')) # 영어 불용어 set
for words in df_nlp['review_split']:
  words=[w for w in words if not w in stops]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
